# Analyzing the Relationship between Crime Rates and Alcohol Availability in San Diego County

### Our Question
What is the correlation between the crime rates and alcohol availability in San Diego County? Is there a relationship between the rates of particular crimes (ie. drug/alcohol violations, theft/larceny, DUI, vandalism, sex crimes, etc) and the number of establishments holding ABC licenses in the community/district?

### Why this Question?
Alcohol is arguably the most socially acceptable drug in San Diego; however, it has the potential to negatively impact individuals and communities in various ways. One of the most important of these is crime, and that leads us to wonder if alcohol abundance or restriction might impact its abuse and interrelation with crime.
According to the National Council on Alcoholism and Drug Dependence, "alcohol and drugs are implicated in an estimated 80% of offenses leading to incarceration in the United States such as domestic violence, driving while intoxicated, property offenses, drug offenses, and public-order offenses." We want to examine how San Diego county is affected by the legal availability of alcohol (ie. from liquor markets, grocery stores, restaurants, hotels) and the rates of crimes in the area (particularly those commonly associated with excessive alcohol use; the Alcohol Rehab Guide lists robbery, sexual assault, child abuses as examples).
References:
1) National Council on Alcoholism and Drug Dependence: https://www.ncadd.org/about-addiction/alcohol-drugs-and-crime
2) Alcohol Rehab Guide on Alcohol-related crime: https://www.alcoholrehabguide.org/alcohol/crimes/

### What data will we use to address this question?

CRIME INCIDENCES IN SD COUNTY:
This dataset contains processed crime incidents for SD county based on data supplied by SANDAG from 2007-2013, partitioned by year. It includes the type of crime (drug/alcohol violations, theft/larceny, DUI, robbery, sex crimes, etc.), neighborhood, community, city, latitude and longitude fields (in addition to various date/time fields which might not pertain to us)
Datasets:
incidents-100k.csv: 100,000 random incidences (over 2007-2013)
incidents-5y.csv: ~ 750K incidences (from years 2008-2012)
Reference: http://data.sandiegodata.org/dataset/clarinova_com-crime-incidents-casnd-7ba4-extract

ABC LICENSES IN SD COUNTY
This dataset ontains all current, valid Alcohol and Beverage Control Licenses in the county, with valid latitude and longitude fields
Dataset:
abs-licenses-casnd.csv
Contains 4988 entries
Reference: http://data.sandiegodata.org/dataset/clarinova_com-alcohol_licenses-casnd-429e-extract


We begin by importing several modules. 

In [1]:
% reset
% config InlineBackend.figure_format = 'retina'

%matplotlib inline

import geocoder
import folium
from haversine import haversine

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind
from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 6
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Tahoma']
sns.set(style='whitegrid', context ='paper')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
df_crimes = pd.read_csv('incidents-100k.csv')
df_crimes.head()

,id,date,year,month,day,week,dow,time,hour,is_night,...,comm_pop,council,coun_pop,asr_zone,lampdist,lat,lon,desc,gctype,gcquality
0,NaN,2008-09-19,2008,9,3184,37,5,13:00:00,13,0,...,42779,San005,143957,1,3162,32.957337,-117.143777,ILLEGAL POSSESS TEAR GAS/ETC,cns/segment,54
1,NaN,2008-07-20,2008,7,3123,28,0,22:00:00,22,1,...,15748,San003,147116,3,8000,32.758153,-117.125022,FRAUD,cns/segment,65
2,NaN,2009-05-25,2009,5,3432,21,1,00:30:00,0,1,...,39849,San002,142456,6,2236,32.796761,-117.254577,BATTERY WITH SERIOUS BODILY INJURY,cns/segment,65
3,NaN,2011-04-22,2011,4,4129,16,5,19:10:00,19,0,...,45697,San003,147116,6,1414,32.757287,-117.129870,POSSESS NARCOTIC CONTROLLED SUBSTANCE,cns/segment,65
4,NaN,2012-05-12,2012,5,4515,19,6,21:40:00,21,1,...,77873,San006,140738,1,6082,32.836098,-117.206645,BATTERY WITH SERIOUS BODILY INJURY (F),cns/segment,65


In [3]:
df_crimes = df_crimes.drop(['id', 'dow', 'hour', 'gctype', 'address','gcquality', 'lampdist', 'desc'], axis=1)
df_crimes.head()

,date,year,month,day,week,time,is_night,type,city,segment_id,nbrhood,community,comm_pop,council,coun_pop,asr_zone,lat,lon
0,2008-09-19,2008,9,3184,37,13:00:00,0,WEAPONS,SndSAN,46189,SanRNC,SanRNC,42779,San005,143957,1,32.957337,-117.143777
1,2008-07-20,2008,7,3123,28,22:00:00,1,FRAUD,SndSAN,4410,SanNRM,SanMDS,15748,San003,147116,3,32.758153,-117.125022
2,2009-05-25,2009,5,3432,21,00:30:00,1,ASSAULT,SndSAN,30948,SanPCF,SanPCF,39849,San002,142456,6,32.796761,-117.254577
3,2011-04-22,2011,4,4129,16,19:10:00,0,DRUGS/ALCOHOL VIOLATIONS,SndSAN,45374,SanNRT,SanGRE,45697,San003,147116,6,32.757287,-117.129870
4,2012-05-12,2012,5,4515,19,21:40:00,1,ASSAULT,SndSAN,45771,SanNOA,SanCLA,77873,San006,140738,1,32.836098,-117.206645


In [7]:
df_alcohol = pd.read_csv('abc_licenses_sdcounty.csv')
df_alcohol['premisesaddress']

0                 1215 EL CARMEL PL, SAN DIEGO, CA  92109
1               9349 MIRA MESA BLVD, SAN DIEGO, CA  92126
2              3744 OCEAN VIEW BLVD, SAN DIEGO, CA  92113
3               3515 UNIVERSITY AVE, SAN DIEGO, CA  92104
4                   1833 GARNET AVE, SAN DIEGO, CA  92109
5                  3787 INGRAHAM ST, SAN DIEGO, CA  92109
6                    4210 MARKET ST, SAN DIEGO, CA  92102
7        110 NATIONAL CITY BLVD, NATIONAL CITY, CA  91950
8                 3263 CAMINO DEL MAR, DEL MAR, CA  92014
9              3638 OCEAN VIEW BLVD, SAN DIEGO, CA  92113
10            25721 JESMOND DENE RD, ESCONDIDO, CA  92026
11                302 W MISSION AVE, ESCONDIDO, CA  92025
12                       210 W F ST, ENCINITAS, CA  92024
13                   3848 CENTRE ST, SAN DIEGO, CA  92103
14                     6 HENSLEY ST, SAN DIEGO, CA  92102
15      15500 SAN PASQUAL VALLEY RD, SAN DIEGO, CA  92027
16            2606 N MISSION BAY DR, SAN DIEGO, CA  92109
17            

In [15]:
#g = geocoder.google(df_alcohol['address'][0])
df_alcohol = df_alcohol.loc[(df_alcohol["premisesaddress"].str.contains(', SAN DIEGO, CA'))] 
df_alcohol['premisesaddress'].head(50)
print(len(df_alcohol))
#df_alcohol = df_alcohol[', SAN DIEGO, CA' in df_alcohol['premisesaddress']]

2661
